<a href="https://colab.research.google.com/github/Mtab-cyber/CD-Lab/blob/main/PCSE23_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
ACCESS_TOKEN="1504889196539834368-pNnsxjfSebxUE2raPXcNLENvewmte0"
ACCESS_TOKEN_SECRET="P1NIV9kpBFT2GVEHpT6n7r0A6mKmms1i8vah6LQPitNfu"
CONSUMER_KEY="xazSVEcvwzMuqzPh1GAE2oydh"
CONSUMER_SECRET="WR0jPEWtTZSaoeJknB18YqoBfn6GZ2UbIQsX5qR6BnEqqtm4Zz"

In [10]:
from tweepy import API
from tweepy import OAuthHandler
from io import BytesIO
import base64
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import re

In [11]:
class TwitterAuthenticator():
  def authenticate_twitter_app(self):
    auth = OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN,ACCESS_TOKEN_SECRET)
    return auth;

In [12]:
class TwitterClient():
  def _init_(self,twitter_user=None):
    self.auth = TwitterAuthenticator().authenticate_twitter_app()
    self.twitter_client = API(self.auth)
    self.twitter_user = twitter_user
  
  def get_twitter_client_api(self):
    return self.twitter_client


In [13]:
class TweetAnalyzer():
    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        if analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
            return -1

    def tweets_to_data_frame(self, tweets):  
        df = pd.DataFrame([tweet.full_text for tweet in tweets], columns=['tweets'])

        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.full_text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])
        df['sentiment'] = np.array([self.analyze_sentiment(tweet) for tweet in df['tweets']])
        df['polarity']=np.array([self.Polarity(tweet) for tweet in df['tweets']])
        df['subjectivity']=np.array([self.subjectivity(tweet) for tweet in df['tweets']])
        return df

    def Polarity(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        polarity = analysis.sentiment.polarity 
        return polarity
        
    
    def subjectivity(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        subjectivity = analysis.sentiment.subjectivity
        return subjectivity

In [14]:
class keyword():
    def key(word):
        twitter_client = TwitterClient()
        tweet_analyzer = TweetAnalyzer()
        api = twitter_client.get_twitter_client_api()
        tweets = api.user_timeline(screen_name=word, count=200, tweet_mode='extended')
        df = tweet_analyzer.tweets_to_data_frame(tweets)
        return df

In [15]:
class plotting():
    def show_wordcloud(data, title = None):
        img = BytesIO()
        stopwords = set(STOPWORDS)
        wd = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=200,
        max_font_size=40, 
        scale=3,
        random_state=1
        ).generate(str(data))
        
        plt.axis('off')
        plt.imshow(wd,interpolation='bilinear')
        wd.to_image().save(img, 'JPEG')
        img.seek(0)
        plotwrd = base64.b64encode(img.getvalue())
        plt.close()
        return plotwrd.decode('utf-8')
    
    def sentiment(wrd):
        data=[]
        for sent in range(0,len(wrd)):
            if wrd['sentiment'][sent]>0:
                strng='Positive'
                data.append(strng)
            elif wrd['sentiment'][sent]==0:
                strng='Neutral'
                data.append(strng)
            else:
                strng='Negative'
                data.append(strng)
        dd=pd.DataFrame(data,columns=["sentiments"])
        sns.catplot(x="sentiments", kind="count", palette="Blues_d", data=dd)
        img = BytesIO()
        plt.savefig(img, format='png')
        img.seek(0) 
        plotsenti = base64.b64encode(img.getvalue())
        plt.close()
        return plotsenti.decode('utf-8')
        

    def PolarityAndSubjectivity(df):
        plt.rcParams['figure.figsize'] = [10, 8]
        for index, tweets in enumerate(df.index):
            x = df.polarity.loc[tweets]
            y = df.subjectivity.loc[tweets]
            plt.scatter(x, y, color='Red')
            plt.title('Sentiment Analysis', fontsize = 20)
            plt.xlabel('Polarity', fontsize=15)
            plt.ylabel('Subjectivity', fontsize=10)
        img2 = BytesIO()
        plt.tight_layout()
        plt.savefig(img2, format='jpeg',dpi=500)
        img2.seek(0)
        plot_url = base64.b64encode(img2.getvalue())
        
        return plot_url.decode('utf-8')

Twitter App using Flask

In [18]:
from flask import Flask, request, render_template

In [20]:
app = Flask("Project")

In [21]:
@app.route('/')
def home():
  return render_template('Twitter.html')

In [ ]:
@app.route('/keyword',methods=['POST','GET'])
def keyword():
  val=request.from["keyword"]
  word=keyword.key(val)
  polarity=plotting.PolarityAndSubjectivity(word)
  word_cld=plotting.show_wordcloud(word['tweets'])
  negative_positive_plot=plotting.sentiment(word)
  return render_template('Twitter.html',prediction=word['tweets'].head(),Top_Five_Tweets="Top Five Raw Tweets:",plot=word_cld,Word_cloud="Word Cloud for common words used in the Tweets",plot_polarity=polarity,polatiry_plot="Relationship between Subjectivity and Polarity",plot_sentiment=negative_positive_plot,Sentiment="Count of Positive, Negative and Neutral Tweets")

if __name__ == "__main__":
    app.run()